In [59]:
import pandas as  pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_validate
reviews = pd.read_json('dataset_gamereactor_reviews.json')
stopwords = pd.read_json('stop_words_swedish.json')
reviews

,text,headline,publisher,comments,date,rating
0,"Patrik har invaderat en främmande planet, mörd...",The Riftbreaker,Patrik Severin,5,2021-11-02,7
1,Måns har snörat på sig pjäxorna och gränslat s...,Riders Republic,Måns Lindman,9,2021-10-31,8
2,Mario Party-serien återvänder till sina rötter...,Mario Party Superstars,Jonas Mäki,14,2021-10-28,8
3,Vi har återvänt till 1993 för att under sommar...,Echo Generation,Jonas Mäki,0,2021-10-27,7
4,Rymdens väktare är tillbaka och Henric har gjo...,Marvel's Guardians of the Galaxy,Henric Pettersson,13,2021-10-25,8
...,...,...,...,...,...,...
4490,Gamereactor håller redan nu tummarna för att ä...,Legend of Zelda: The Wind Waker,Jonas Mäki,93,2003-05-07,9
4491,Novalogic förflyttar sina trogna trupper till ...,Delta Force: Black Hawk Down,Petter Hegevall,13,2003-05-07,8
4492,Det är enligt mig vansinnigt konstigt att näst...,Batman: Dark Tomorrow,Petter Hegevall,20,2003-05-07,2
4493,Konami återvänder till staden vi avskyr att äl...,Silent Hill 3,Gamereactor,25,2003-05-07,9


In [60]:
reviews['ratingeasy'] = reviews['rating'].apply(lambda x: 0 if x < 6 else 1)
reviews

,text,headline,publisher,comments,date,rating,ratingeasy
0,"Patrik har invaderat en främmande planet, mörd...",The Riftbreaker,Patrik Severin,5,2021-11-02,7,1
1,Måns har snörat på sig pjäxorna och gränslat s...,Riders Republic,Måns Lindman,9,2021-10-31,8,1
2,Mario Party-serien återvänder till sina rötter...,Mario Party Superstars,Jonas Mäki,14,2021-10-28,8,1
3,Vi har återvänt till 1993 för att under sommar...,Echo Generation,Jonas Mäki,0,2021-10-27,7,1
4,Rymdens väktare är tillbaka och Henric har gjo...,Marvel's Guardians of the Galaxy,Henric Pettersson,13,2021-10-25,8,1
...,...,...,...,...,...,...,...
4490,Gamereactor håller redan nu tummarna för att ä...,Legend of Zelda: The Wind Waker,Jonas Mäki,93,2003-05-07,9,1
4491,Novalogic förflyttar sina trogna trupper till ...,Delta Force: Black Hawk Down,Petter Hegevall,13,2003-05-07,8,1
4492,Det är enligt mig vansinnigt konstigt att näst...,Batman: Dark Tomorrow,Petter Hegevall,20,2003-05-07,2,0
4493,Konami återvänder till staden vi avskyr att äl...,Silent Hill 3,Gamereactor,25,2003-05-07,9,1


In [61]:
text_train, y_train, y_traineasy = reviews['text'], reviews['rating'], reviews['ratingeasy']
vectorizer = CountVectorizer()
X_train = vectorizer.fit_transform(text_train)
X_train.shape



(4495, 16097)

In [62]:

def perform_cv(X, y, name):
    cv_results = cross_validate(LogisticRegression(random_state=42), X, y, cv=5, 
                                scoring=['accuracy', 'f1_weighted'], n_jobs=-1)
    print(f"Mean cross-validation accuracy ({name}):", cv_results['test_accuracy'].mean())
    print(f"Mean cross-validation F1-score ({name}):", cv_results['test_f1_weighted'].mean())

perform_cv(X_train, y_train, "original ratings")

perform_cv(X_train, y_traineasy, "binary ratings")

C_values = [0.001, 0.01, 0.1, 1, 10, 100]
best_score_train, best_score_traineasy = 0, 0
best_C_train, best_C_traineasy = 0, 0

for C in C_values:
    lr = LogisticRegression(C=C, random_state=42)
    score_train = cross_validate(lr, X_train, y_train, cv=5, scoring='accuracy', n_jobs=-1)['test_score'].mean()
    score_traineasy = cross_validate(lr, X_train, y_traineasy, cv=5, scoring='accuracy', n_jobs=-1)['test_score'].mean()
    
    if score_train > best_score_train:
        best_score_train, best_C_train = score_train, C
    if score_traineasy > best_score_traineasy:
        best_score_traineasy, best_C_traineasy = score_traineasy, C

print(f"\nBest C for original ratings: {best_C_train}, Score: {best_score_train}")
print(f"Best C for binary ratings: {best_C_traineasy}, Score: {best_score_traineasy}")

/home/m/dev/ai/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Mean cross-validation accuracy (original ratings): 0.22091212458286985
Mean cross-validation F1-score (original ratings): 0.1972285772654566
Mean cross-validation accuracy (binary ratings): 0.7432703003337041
Mean cross-validation F1-score (binary ratings): 0.6984979019972567


/home/m/dev/ai/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/m/dev/ai/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_ite


Best C for original ratings: 0.001, Score: 0.2589543937708565
Best C for binary ratings: 0.1, Score: 0.7563959955506118


In [63]:
stopwords_list = stopwords[0].tolist()

cv_with_stop_words = CountVectorizer(stop_words=stopwords_list)
X_train_with_stop_words = cv_with_stop_words.fit_transform(text_train)
n_features_with_stop_words = X_train_with_stop_words.shape[1]

print(f"Number of features with stop words: {n_features_with_stop_words}")
print(f"Number of words cut: {X_train.shape[1] - n_features_with_stop_words}")
perform_cv(X_train_with_stop_words, y_train, "original ratings (with stop words)")
perform_cv(X_train_with_stop_words, y_traineasy, "binary ratings (with stop words)")
best_score_train, best_score_traineasy = 0, 0
best_C_train, best_C_traineasy = 0, 0

for C in C_values:
    lr = LogisticRegression(C=C, random_state=42)
    score_train = cross_validate(lr, X_train_with_stop_words, y_train, cv=5, scoring='accuracy', n_jobs=-1)['test_score'].mean()
    score_traineasy = cross_validate(lr, X_train_with_stop_words, y_traineasy, cv=5, scoring='accuracy', n_jobs=-1)['test_score'].mean()
    
    if score_train > best_score_train:
        best_score_train, best_C_train = score_train, C
    if score_traineasy > best_score_traineasy:
        best_score_traineasy, best_C_traineasy = score_traineasy, C

print(f"\nBest C for original ratings (with stop words): {best_C_train}, Score: {best_score_train}")
print(f"Best C for binary ratings (with stop words): {best_C_traineasy}, Score: {best_score_traineasy}")

Number of features with stop words: 15785
Number of words cut: 312
Mean cross-validation accuracy (original ratings (with stop words)): 0.2264738598442714
Mean cross-validation F1-score (original ratings (with stop words)): 0.19926521203575825
Mean cross-validation accuracy (binary ratings (with stop words)): 0.7399332591768631
Mean cross-validation F1-score (binary ratings (with stop words)): 0.6905554804166278

Best C for original ratings (with stop words): 0.001, Score: 0.2593993325917686
Best C for binary ratings (with stop words): 0.1, Score: 0.753948832035595


In [64]:
tfidf = TfidfVectorizer(stop_words=stopwords_list)
X_train_tfidf = tfidf.fit_transform(text_train)
n_features_tfidf = X_train_tfidf.shape[1]

print(f"Number of features with TF-IDF: {n_features_tfidf}")
perform_cv(X_train_tfidf, y_train, "original ratings (TF-IDF)")
perform_cv(X_train_tfidf, y_traineasy, "binary ratings (TF-IDF)")

best_score_train_tfidf, best_score_traineasy_tfidf = 0, 0
best_C_train_tfidf, best_C_traineasy_tfidf = 0, 0

for C in C_values:
    lr = LogisticRegression(C=C, random_state=42)
    score_train = cross_validate(lr, X_train_tfidf, y_train, cv=5, scoring='accuracy', n_jobs=-1)['test_score'].mean()
    score_traineasy = cross_validate(lr, X_train_tfidf, y_traineasy, cv=5, scoring='accuracy', n_jobs=-1)['test_score'].mean()
    
    if score_train > best_score_train_tfidf:
        best_score_train_tfidf, best_C_train_tfidf = score_train, C
    if score_traineasy > best_score_traineasy_tfidf:
        best_score_traineasy_tfidf, best_C_traineasy_tfidf = score_traineasy, C

print(f"\nBest C for original ratings (TF-IDF): {best_C_train_tfidf}, Score: {best_score_train_tfidf}")
print(f"Best C for binary ratings (TF-IDF): {best_C_traineasy_tfidf}, Score: {best_score_traineasy_tfidf}")

# Compare TF-IDF scores with previous results
print("\nComparison of scores:")
print(f"Original ratings - Count Vectorizer: {best_score_train}, TF-IDF: {best_score_train_tfidf}")
print(f"Binary ratings - Count Vectorizer: {best_score_traineasy}, TF-IDF: {best_score_traineasy_tfidf}")

# Analyze highest and lowest TF-IDF values
feature_names = tfidf.get_feature_names_out()
tfidf_scores = X_train_tfidf.toarray().sum(axis=0)

highest_tfidf = sorted(zip(feature_names, tfidf_scores), key=lambda x: x[1], reverse=True)[:10]
lowest_tfidf = sorted(zip(feature_names, tfidf_scores), key=lambda x: x[1])[:10]

print("\nTop 10 words with highest TF-IDF scores:")
for word, score in highest_tfidf:
    print(f"{word}: {score}")

print("\nTop 10 words with lowest TF-IDF scores:")
for word, score in lowest_tfidf:
    print(f"{word}: {score}")


Number of features with TF-IDF: 15785
Mean cross-validation accuracy (original ratings (TF-IDF)): 0.25650723025583977
Mean cross-validation F1-score (original ratings (TF-IDF)): 0.17240678539644855
Mean cross-validation accuracy (binary ratings (TF-IDF)): 0.7510567296996664
Mean cross-validation F1-score (binary ratings (TF-IDF)): 0.6484389604415928


/home/m/dev/ai/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/m/dev/ai/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_ite


Best C for original ratings (TF-IDF): 0.001, Score: 0.2593993325917686
Best C for binary ratings (TF-IDF): 1, Score: 0.7510567296996664

Comparison of scores:
Original ratings - Count Vectorizer: 0.2593993325917686, TF-IDF: 0.2593993325917686
Binary ratings - Count Vectorizer: 0.753948832035595, TF-IDF: 0.7510567296996664

Top 10 words with highest TF-IDF scores:
är: 161.53947221551618
spel: 64.28927045925359
tillbaka: 50.83552731153804
betyg: 50.11344461531504
satt: 40.40173949294502
jonas: 40.3222855774639
petter: 40.24074253359729
år: 38.574674269632226
äntligen: 37.71473827160897
spelet: 37.54266980309155

Top 10 words with lowest TF-IDF scores:
been: 0.12507843901245583
peers: 0.12507843901245583
puttin: 0.12507843901245583
rockin: 0.12507843901245583
suckas: 0.12507843901245583
years: 0.12507843901245583
bioduken: 0.17975480317078033
blodsugare: 0.17975480317078033
bloodlines: 0.17975480317078033
gryningen: 0.17975480317078033
